In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate
import pandas as pd
from data_clean import *
import atel
from atel.data import BookCollection

# HuggingFace Trainer

In [2]:
import torch

In [3]:
t = torch.tensor([0.5, 0.94, 0.3])

In [5]:
TARGET     = 'Perspektiv'
set_seed(42)

book_col = BookCollection(data_file="./data/book_col_271120.pkl")
df, labels = get_pandas_dataframe(book_col, TARGET)

NUM_LABELS = len(labels)

Seed has been set to 42
Loaded from disk: ./data/book_col_271120.pkl
(778,)
1


In [6]:
NUM_LABELS

2

In [7]:
df.dtypes

text       object
labels    float64
dtype: object

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Maltehb/danish-bert-botxo")
model = AutoModelForSequenceClassification.from_pretrained("Maltehb/danish-bert-botxo",
                                                           num_labels=NUM_LABELS,
                                                           problem_type='multi_label_classification')

Some weights of the model checkpoint at Maltehb/danish-bert-botxo were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
t = Dataset.from_pandas(df)

In [10]:
t2 = t.map(tokenize_function, batched=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.78ba/s]


In [11]:
metric = evaluate.load("accuracy")

In [12]:
def compute_metrics_multilabel(eval_pred):
    logits, labels = eval_pred
    return {"accuracy": acc_metric(torch.tensor(logits), torch.tensor(labels))}

In [13]:
t2

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 778
})

In [67]:
torch.tensor([t2['labels'][0]])

tensor([[0., 0., 1., 0., 0.]])

In [99]:
from torchmetrics.classification import MultilabelAccuracy
from torchmetrics import Accuracy
acc = MultilabelAccuracy(num_labels=NUM_LABELS, average='micro')
acc2 = Accuracy(subset_accuracy=True)

In [100]:
preds = torch.tensor([[0.0944,  0.1316, -0.4613, -0.1432, -0.2011]])

In [101]:
preds = torch.tensor([[0.944,  -0.316, 0.4613, -0.432, -0.4011]])

In [102]:
sm(preds)

tensor([[0.7199, 0.4217, 0.6133, 0.3936, 0.4010]])

In [108]:
acc = MultilabelAccuracy(num_labels=3, average='micro')
preds = torch.tensor([[0.9, 0.1, 0.9], [0.1, 0.1, 0.9]])
target = torch.tensor([[1, 0, 1], [0, 0, 1]])
acc2(preds, target)

tensor(1.)

In [95]:
preds.shape

torch.Size([2, 3])

In [98]:
acc2(preds, target)

tensor(0.5000)

In [51]:
ls = torch.nn.CrossEntropyLoss()
ls(torch.tensor([-0.4755,  0.0560, -1.3267,  0.3594, -0.0170, -0.5276]), torch.tensor(t2['labels'][0]).long())

RuntimeError: size mismatch (got input: [6], target: [5])

In [56]:
sm = torch.nn.Sigmoid()
sm(torch.tensor([ 0.0944,  0.1316, -0.4613, -0.1432, -0.2011]))

tensor([0.5236, 0.5329, 0.3867, 0.4643, 0.4499])

In [76]:
torch.tensor([t2['labels'][0]])

tensor([[0., 0., 1., 0., 0.]])

In [50]:
model(input_ids=torch.tensor([t2['input_ids'][0]]), labels=torch.tensor([t2['labels'][0]]))

SequenceClassifierOutput(loss=tensor(0.7349, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0944,  0.1316, -0.4613, -0.1432, -0.2011]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [17]:
training_args = TrainingArguments(
    output_dir="test_trainer", 
    save_strategy='no', 
    evaluation_strategy='epoch',
    report_to='none',
    num_train_epochs=1
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=t2,
    compute_metrics=compute_metrics_multilabel
)

RuntimeError: CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 6.00 GiB total capacity; 5.29 GiB already allocated; 0 bytes free; 5.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.train()

# Native PyTorch

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [ ]:
tokenized_datasets.set_format("torch")

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)